# step1:处理数据

In [6]:
import os  
import torch  
import pandas as pd  
import matplotlib.pyplot as plt
  
    #step1:加载数据集
# 设置数据集的路径  
data_dir = '../DataFiles/ODS/strain'   
# 创建空的列表来存储所有的DataFrame  
X_data0 = []  
y_data0 = []
# 遍历指定目录下的所有文件  
for filename in os.listdir(data_dir):  
    if filename.endswith('.csv'):  
        # 构建文件的完整路径  
        file_path = os.path.join(data_dir, filename)  
        # 读取Excel文件  
        df = pd.read_csv(file_path)
        serie=df.iloc[:, 0]
        version_without_extension = filename.rsplit(".", 1)[0] 
        serie.name=version_without_extension
        # 将读取的DataFrame添加到列表中  
        X_data0.append(serie)
        y_data0.append(version_without_extension)

    #step2:遍历X_data0中的每个DataFrame，并删除非数字数据  
cleaned_data = []  
for df in X_data0:  
    df_cleaned = df[pd.to_numeric(df, errors='coerce').notnull()]  
    cleaned_data.append(df_cleaned)  
 

    #step3:将X_data0中的每个DataFrame的所有元素变为float类型 
for i, df in enumerate(cleaned_data):  
    X_data0[i] = df.astype(float) 

# 初始化X_data1来存储结果  
X_data1 = []  
  
# 遍历X_data0中的每个Series  
for series in X_data0:  
    # 计算步长（由于重叠率为50%，步长为序列长度的一半）  
    step_size = 1024  # 因为2048的一半是1024，且我们想要50%的重叠  
    # 初始化一个空的list来存储子序列  
    subsequences = []  
    # 使用range和步长来生成子序列  
    for i in range(0, len(series) - 2048+ 1, step_size):  
        # 确保切片是一维的，并转换为Series（虽然这里切片已经是Series）  
        subsequence = series.iloc[i:i+2048]  
        subsequences.append(subsequence) 
    X_data1.append(subsequences)  
type(X_data1),len(X_data1),len(X_data1[0])

(list, 2, 1814)

In [7]:
import pandas as pd  
import numpy as np  
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import LabelEncoder  
from keras.utils import to_categorical
import time  
from keras.callbacks import Callback  
import torch 

# 假设 X_data1 已经定义好了  
all_data = []  
all_labels = []  
  
for sublist in X_data1:  
    for series in sublist:  
        # 假设每个series是pandas Series  
        all_data.append(series.values)  
        all_labels.append(series.name)  
  
# 将数据转换为numpy数组  
data_array = np.array(all_data)  
# 将标签转换为编码  

label_encoder = LabelEncoder()  
labels_encoded = label_encoder.fit_transform(all_labels)  
  
# 如果需要one-hot编码  
labels_one_hot = to_categorical(labels_encoded)  
  
# 这里你可能需要reshape data_array 以符合LSTM的输入要求  
# LSTM通常要求输入形状为 (samples, time_steps, features)  
# 假设每个series的长度相同，或者你可以填充/截断它们以具有相同的长度  
# time_steps = data_array.shape[1]  # 假设所有series长度相同  
# data_array = data_array.reshape(-1, time_steps, 1)  # 增加一个维度以匹配LSTM的输入要求

# 分割数据集  
X_train, X_temp, y_train, y_temp = train_test_split(data_array, labels_one_hot, test_size=0.3, random_state=42)  
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


X_train = torch.tensor(X_train, dtype=torch.float32)  
y_train = torch.tensor(y_train, dtype=torch.float32)  

X_val = torch.tensor(X_val, dtype=torch.float32)  
y_val = torch.tensor(y_val, dtype=torch.float32) 

X_test = torch.tensor(X_test, dtype=torch.float32)  
y_test = torch.tensor(y_test, dtype=torch.float32) 
type(X_train),X_train.shape,type(y_train),y_train.shape

# 如果您的数据不适合内存中的批处理，您可能需要使用 DataLoader  
# 但为了简单起见，这里我们手动进行批处理  
from torch.utils.data import TensorDataset, DataLoader  
batch_size=1
traindataset = TensorDataset(X_train, y_train)  
trainloader = DataLoader(traindataset, batch_size=batch_size, shuffle=True)  

valdataset = TensorDataset(X_val, y_val)  
valloader = DataLoader(valdataset, batch_size=batch_size, shuffle=True) 

testdataset = TensorDataset(X_test, y_test)  
testloader = DataLoader(testdataset, batch_size=batch_size, shuffle=True) 

# step2:构建小波卷积层(用的是离散小波变换)
- 利用离散小波函数的高通滤波器(一维窗口)和低通滤波器(一维窗口)各自作为卷积核，获得两个原始信号在各自滤波器的投影一维序列；
- 高通滤波器的投影一维序列包含了高频信号部分，多是以细节和噪音信息为主；
- 低通滤波器的投影一维序列包含了低频信号部分，多是以大致趋势为主；
- 待估参数是两个滤波器中的数值；

In [8]:
import pywt
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial

def create_wavelet_filter(wave, in_size, out_size, type=torch.float):
    # 创建一维小波对象
    w = pywt.Wavelet(wave)
    # 生成降采样滤波器
    dec_hi = torch.tensor(w.dec_hi[::-1], dtype=type)
    dec_lo = torch.tensor(w.dec_lo[::-1], dtype=type)
    dec_filters = torch.stack([dec_lo.unsqueeze(0),
                               dec_hi.unsqueeze(0)], dim=0)
    dec_filters = dec_filters.repeat(in_size,1,1)
    # 生成重建滤波器
    rec_hi = torch.tensor(w.rec_hi[::-1], dtype=type).flip(dims=[0])
    rec_lo = torch.tensor(w.rec_lo[::-1], dtype=type).flip(dims=[0])
    rec_filters = torch.stack([rec_lo.unsqueeze(0),
                               rec_hi.unsqueeze(0)], dim=0)
    rec_filters = rec_filters.repeat(out_size, 1, 1)
    return dec_filters, rec_filters
import torch
import torch.nn.functional as F

def wavelet_transform(x, filters):
    # x 的形状应为 (b, c, w) 其中 w 是宽度（或序列长度）
    b, c, w = x.shape
    x = F.conv1d(x, filters, stride=2)
    x = x.reshape(b, c, 2, w // 2)
    return x

def inverse_wavelet_transform(x, filters):
    # x 的形状应为 (b, c * 4, w_half)
    b, c, _, w_half = x.shape
    pad = (filters.shape[2] // 2 - 1,)
    x = x.reshape(b, c * 2, w_half)
    x = F.conv_transpose1d(x, filters, stride=2, groups=c, padding=pad)
    return x
class _ScaleModule(nn.Module):
    def __init__(self, dims, init_scale=1.0, init_bias=0):
        super(_ScaleModule, self).__init__()
        self.dims = dims
        self.weight = nn.Parameter(torch.ones(*dims) * init_scale)
        self.bias = None

    def forward(self, x):
        return torch.mul(self.weight, x)

import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial

class WTConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1, bias=True, wt_levels=1, wt_type='db1'):
        super(WTConv1d, self).__init__()
        assert in_channels == out_channels

        self.in_channels = in_channels
        self.wt_levels = wt_levels
        self.stride = stride
        self.dilation = 1#即卷积核元素之间没有间距

        # 生成的滤波器
        self.wt_filter, self.iwt_filter = create_wavelet_filter(wt_type, in_channels, in_channels, torch.float)
        #nn.Parameter 是一个特殊的 Tensor，它被注册为模型的一个参数，并会在训练过程中自动更新。
        #requires_grad=False 表示这个参数在训练过程中不需要计算梯度，因此不会被优化算法更新。
        self.wt_filter = nn.Parameter(self.wt_filter, requires_grad=False)
        self.iwt_filter = nn.Parameter(self.iwt_filter, requires_grad=False)
        # functools.partial 函数，它的作用是创建一个新的函数，该函数已经预先填充了一些参数
        self.wt_function = partial(wavelet_transform, filters=self.wt_filter)
        self.iwt_function = partial(inverse_wavelet_transform, filters=self.iwt_filter)

        # 基础卷积和缩放模块
        self.base_conv = nn.Conv1d(in_channels, in_channels, kernel_size, padding='same', stride=1, dilation=1, groups=in_channels, bias=bias)
        self.base_scale = _ScaleModule([1, in_channels, 1])

        # 小波卷积层和缩放
        self.wavelet_convs = nn.ModuleList(
            [nn.Conv1d(in_channels * 2, in_channels * 2, kernel_size, padding='same', stride=1, dilation=1, groups=in_channels * 2, bias=False) for _ in range(self.wt_levels)]
        )
        self.wavelet_scale = nn.ModuleList(
            [_ScaleModule([1, in_channels * 2, 1], init_scale=0.1) for _ in range(self.wt_levels)]
        )

        if self.stride > 1:
            self.stride_filter = nn.Parameter(torch.ones(in_channels, 1, 1), requires_grad=False)
            self.do_stride = lambda x_in: F.conv1d(x_in, self.stride_filter, bias=None, stride=self.stride, groups=in_channels)
        else:
            self.do_stride = None

    def forward(self, x):
        x_ll_in_levels = []
        x_h_in_levels = []
        shapes_in_levels = []

        curr_x_ll = x.unsqueeze(1)
        for i in range(self.wt_levels):
            curr_shape = curr_x_ll.shape
            shapes_in_levels.append(curr_shape)
            #如果当前层级的宽度不是偶数，使用 F.pad 函数对 curr_x_ll 进行填充，以保证宽度是偶数。
            if curr_shape[2] % 2 > 0:
                curr_pads = (curr_shape[2] % 2,)
                curr_x_ll = F.pad(curr_x_ll, curr_pads)
            curr_x = self.wt_function(curr_x_ll)
            #从 curr_x 中提取低频部分（第 0 个索引），更新 curr_x_ll。
            #将 curr_x 重塑为 (b, c * 2, w // 2)。
            #通过 self.wavelet_convs[i] 和 self.wavelet_scale[i] 对 curr_x_tag 进行卷积和缩放操作。
            curr_x_ll = curr_x[:, :, 0, :]
            curr_x_h = curr_x[:, :, 1, :]
            x_ll_in_levels.append(curr_x_ll)
            x_h_in_levels.append(curr_x_h)
        
        if self.stride:
            if self.do_stride is not None:
                x_ll_in_levels = [self.do_stride(x) for x in x_ll_in_levels]
                x_h_in_levels = [self.do_stride(x) for x in x_h_in_levels]
        return x_ll_in_levels[0],x_h_in_levels[0]


# step3:构建完整model结构

In [10]:
import torch  
import torch.nn as nn  
import torch.nn.functional as F  
  
class ConvBiLSTMModel(nn.Module):  
    def __init__(self, input_channels=1, hidden_size=64, num_classes=12):  
        super(ConvBiLSTMModel, self).__init__()  
        # 第一层小波卷积  
        self.wtconv = WTConv1d(in_channels=1, out_channels=1)   
        # BiLSTM层  
        self.bilstm_ll = nn.LSTM(input_size=1, hidden_size=hidden_size,  
                              num_layers=1, bidirectional=True,  
                              batch_first=True)  
        self.bilstm_h = nn.LSTM(input_size=1, hidden_size=hidden_size,  
                              num_layers=1, bidirectional=True,  
                              batch_first=True)  
        # 线性层  
        self.fc = nn.Linear(hidden_size * 4, num_classes)  # *2 因为是双向的  
  
    def forward(self, x):  
        # 通过卷积层  
        x_ll,x_h =self.wtconv(x) 
        # 调整维度以匹配LSTM的输入要求 [batch_size, seq_len, num_features]  
        # LSTM需要 (seq_len, batch, input_size) 但我们设置batch_first=True 
        # LSTM的输出是 (output, (h_n, c_n))  
        # output: [batch_size, seq_len, num_directions * hidden_size]  
        # 我们只关心最后一个时间步的输出  
        # 使用permute来交换维度  
        x_ll = x_ll.permute(0, 2, 1)  
        x_h = x_h.permute(0, 2, 1)
        lstm_out_ll, _ = self.bilstm_ll(x_ll)  
        lstm_out_ll = lstm_out_ll[:, -1, :]  # 取最后一个时间步的输出  
        lstm_out_h, _ = self.bilstm_h(x_h)  
        lstm_out_h = lstm_out_h[:, -1, :]  # 取最后一个时间步的输出  
        lstm_out=torch.cat((lstm_out_ll, lstm_out_h), dim=1)
        # 通过全连接层  
        x = self.fc(lstm_out)  
          
        return x  

# 创建模型实例  
model = ConvBiLSTMModel()  
  
# 如果需要，可以添加损失函数和优化器  
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 


# step4:训练模型

In [11]:
import torch  
import torch.nn as nn  
import torch.optim as optim  
import time   
from tqdm import tqdm  # 导入tqdm库
import pickle  

# 训练模型  
num_epochs = 100  # 训练轮次  
start_time = time.time()  
elapsed_time = 0  
  
#训练模型  
list_train_loss=[]
list_train_accuracy=[]
list_val_loss=[]
list_val_accuracy=[]
# 训练循环  
for epoch in range(num_epochs):  
    model.train()  # 设置模型为训练模式  
    train_running_loss = 0.0  # 用于记录当前epoch的训练损失  
    train_correct = 0  # 用于计算准确率的变量  
    train_total = 0  # 总样本数  
    # 使用tqdm包装训练数据加载器以显示进度条  
    with tqdm(trainloader, desc=f'Train Epoch {epoch+1}/{num_epochs}', unit='batch') as t:  
        for batch_idx, (inputs, labels) in enumerate(t):  
            # 前向传播  
            outputs = model(inputs)  
            loss = criterion(outputs, labels)  
            # 计算准确率  
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels, 1)  
            train_total += labels.size(0)  
            train_correct += (predicted == labels).sum().item()
            # 反向传播和优化  
            optimizer.zero_grad()  # 清除之前的梯度  
            loss.backward()        # 反向传播计算梯度  
            optimizer.step()       # 更新参数  
            
            train_running_loss += loss.item()  # 累加训练损失  
            # 更新进度条信息  
            t.set_postfix(loss=loss.item(), accuracy=100. * train_correct / train_total,train_correct=train_correct,train_total=train_total)  
  
    # 计算平均训练损失和准确率
    train_loss = train_running_loss / len(trainloader.dataset) 
    train_accuracy = 100. * train_correct / train_total  
    list_train_loss.append(train_loss)
    list_train_accuracy.append(train_accuracy)
    # 验证模型  
    model.eval()  # 设置模型为评估模式  
    val_running_loss = 0.0  # 用于记录当前epoch的训练损失  
    val_correct = 0  # 用于计算准确率的变量  
    val_total = 0  # 总样本数  
    # 使用tqdm包装训练数据加载器以显示进度条  
    with tqdm(valloader, desc=f'Val Epoch {epoch+1}/{num_epochs}', unit='batch') as v:
        with torch.no_grad():  # 不需要计算梯度  
            for batch_idx, (inputs, labels) in enumerate(v):  
                # 前向传播  
                outputs = model(inputs)  
                loss = criterion(outputs, labels)  
                # 计算准确率  
                _, predicted = torch.max(outputs.data, 1)  
                _, labels = torch.max(labels, 1)  
                val_total += labels.size(0)  
                val_correct += (predicted == labels).sum().item()
                val_running_loss += loss.item()  # 累加训练损失  
                # 更新进度条信息  
                v.set_postfix(loss=loss.item(), accuracy=100. * val_correct / val_total)  
  
    # 计算平均训练损失和准确率
    val_loss = val_running_loss / len(valloader.dataset)  
    val_accuracy = 100. * val_correct / val_total  
    list_val_loss.append(val_loss)
    list_val_accuracy.append(val_accuracy)
    torch.save(model.state_dict(), 'DWTCNNLSTM'+str(epoch)+'.pth')
    # 显示验证损失和所用时间  
    end_time = time.time()  
    elapsed_time = end_time - start_time  
    # 检查是否超过了5小时  
    if elapsed_time > 4 * 60 * 60:  # 5 hours in seconds 
        print('Training time exceeded 5 hours. Saving model and exiting.')  
        torch.save(model.state_dict(), 'DWTCNNLSTM.pth')   
        break  


torch.save(model.state_dict(), 'DWTCNNLSTM.pth')
print('Training complete.')

Train Epoch 1/100:   0%|          | 0/2412 [00:02<?, ?batch/s]

KeyboardInterrupt



In [ ]:
import matplotlib.pyplot as plt 
# 绘制训练loss和验证loss  
plt.plot(list_train_loss, label='train')  
plt.plot(list_val_loss, label='val')  
plt.legend(loc='upper right')  
plt.title('Model loss')  
plt.ylabel('Loss')  
plt.xlabel('Epoch')  
plt.show()  
  
# 绘制训练accuracy和验证accuracy  
plt.plot(list_train_accuracy, label='train')  
plt.plot(list_val_accuracy, label='val')  
plt.legend(loc='lower right')  
plt.title('Model accuracy')  
plt.ylabel('Accuracy')  
plt.xlabel('Epoch')  
plt.ylim(ymin=0, ymax=100)  # 设置y轴的范围为0到1  
plt.show()
len(list_train_accuracy),len(list_val_accuracy)

In [ ]:
from tqdm import tqdm  # 导入tqdm库
model.eval()  # 设置模型为评估模式  
test_running_loss = 0.0  # 用于记录当前epoch的训练损失  
test_correct = 0  # 用于计算准确率的变量  
test_total = 0  # 总样本数  
with tqdm(testloader, desc=f'Test Epoch:', unit='batch') as t:
        with torch.no_grad():  # 不需要计算梯度  
            for batch_idx, (inputs, labels) in enumerate(t):  
                # 前向传播  
                outputs = model(inputs)  
                loss = criterion(outputs, labels)  
                # 计算准确率  
                _, predicted = torch.max(outputs.data, 1)  
                _, labels = torch.max(labels, 1)  
                test_total += labels.size(0)  
                test_correct += (predicted == labels).sum().item()
                test_running_loss += loss.item()  # 累加训练损失  
                # 更新进度条信息  
                t.set_postfix(loss=loss.item(), accuracy=100. * test_correct / test_total)  
  
    # 计算平均训练损失和准确率
test_loss = test_running_loss / len(testloader.dataset)  
test_accuracy = 100. * test_correct / test_total 

test_loss,test_accuracy

In [ ]:
import pickle
print(help(pickle))